In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import pickle

def run(episodes,is_training, render):
    env = gym.make('FrozenLake-v1', map_name="8x8", is_slippery=False, render_mode='human' if render else None)
    
    if(is_training):
        q = np.zeros((env.observation_space.n, env.action_space.n))
    else:
        f = open('frozen_lake8x8.pkl', 'rb')
        q = pickle.load(f)
        f.close()

    learning_rate_a = 0.01
    discount_factor_g = 0.01
    epsilon = 1
    epsilon_decay_rate = 0.0001
    rng = np.random.default_rng()

    reward_per_episode = np.zeros(episodes)

    for i in range(episodes):
        state = env.reset()[0]
        terminated = False
        truncated = False

        while not terminated and not truncated:
            if is_training and rng.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q[state, :])

            new_state, reward, terminated, truncated, _ = env.step(action)

            if(is_training):
                q[state, action] = q[state, action] + learning_rate_a * (
                    reward + discount_factor_g * np.max(q[new_state, :]) - q[state, action]
                )

            state = new_state

        epsilon = max(epsilon - epsilon_decay_rate, 0)
        if epsilon == 0:
            learning_rate_a = 0.0001

        if reward == 1:
            reward_per_episode[i] = 1

    # Plotting after all episodes
    sum_rewards = np.zeros(episodes)
    for t in range(episodes):
        sum_rewards[t] = np.sum(reward_per_episode[max(0, t-100):(t+1)])
    plt.plot(sum_rewards)
    plt.xlabel('Episode')
    plt.ylabel('100-episode Reward Sum')
    plt.title('FrozenLake 8x8 Performance')
    plt.savefig('frozen_lake8x8.png')
    plt.close()

    # Save Q-table
    if(is_training):
        with open("frozen_lake8x8.pkl", "wb") as f:
            pickle.dump(q, f)

    env.close()

if __name__ == '__main__':
    run(15000, is_training = True, render = False)
